In [1]:
from transfer_learning_publication.data import CaravanDataSource
from transfer_learning_publication.transforms import PipelineBuilder, Log, ZScore
from transfer_learning_publication.containers import TimeSeriesCollection

In [2]:
caravan = CaravanDataSource(
    base_path="/Users/cooper/Desktop/CARAVAN_CLEAN/train",
    region="tajikkyrgyz"
)

basins = caravan.list_gauge_ids()

ts_data = caravan.get_timeseries(
    gauge_ids=basins[:2],
    columns=["streamflow", "total_precipitation_sum"],
)

static_data = caravan.get_static_attributes(
    gauge_ids=basins[:2],
    columns=["area", "high_prec_freq"],
)

In [3]:
static_collection = caravan.to_time_series_collection(ts_data)
static_collection.feature_indices

{'streamflow': 0, 'total_precipitation_sum': 1}

In [ ]:
preprocessing_pipelines = (
    PipelineBuilder(group_identifier="gauge_id")
    .add_per_basin(Log(), columns=["streamflow"])
    .add_global(ZScore(), columns=["streamflow", "total_precipitation_sum"])
    .build()
)

data_transformed = preprocessing_pipelines.fit_transform(ts_data.collect())

data_inv = preprocessing_pipelines.inverse_transform(data_transformed)

NameError: name 'PipelineBuilder' is not defined

In [2]:
ts_data.collect().head(), data_transformed.head(), data_inv.head()

NameError: name 'ts_data' is not defined

In [ ]:
caravan.write_timeseries(
    data_transformed,
    "/Users/cooper/Desktop/transfer-learning-publication/data",
    True
)